<a href="https://colab.research.google.com/github/bojunchen3/Project/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install larq
!pip uninstall sklearn
!pip install pretty_confusion_matrix
%mkdir dataset
%cd /content/drive/MyDrive/project
!unrar x /content/drive/MyDrive/project/dataset.rar -d /content/dataset

In [ ]:
from keras.layers import Softmax, Dropout, Flatten, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from larq.layers import QuantConv2D
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
import larq as lq
import os
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
# 導入資料集
train = ImageDataGenerator(rescale=1/255)
val = ImageDataGenerator(rescale=1/255)
train_dataset = train.flow_from_directory('/content/dataset/train/',
                      target_size=(46,46),
                      batch_size=128,
                      color_mode='grayscale',
                      class_mode='categorical')
val_dataset = val.flow_from_directory('/content/dataset/validation/',
                    target_size=(46,46),
                    batch_size=128,
                    color_mode='grayscale',
                    class_mode='categorical')
kwargs = dict(kernel_quantizer="ste_sign",
        kernel_constraint="weight_clip")
# 定義模型架構
model = Sequential()
model.add(QuantConv2D(filters=8, kernel_size=(3, 3), input_shape=(46, 46, 1), **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=16, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=32, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=32, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=32, kernel_size=(1, 1), input_quantizer="ste_sign", **kwargs))
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=10, kernel_size=(1, 1), input_quantizer="ste_sign", activation='softmax', **kwargs))
model.add(Flatten())

adam = Adam(learning_rate=0.005)
model.compile(optimizer=adam,
       loss='categorical_crossentropy',
       metrics=['accuracy'])
# 定義學習率調整函數
def lr_scheduler(epoch, learning_rate):
    if epoch > 0 and epoch % 20 ==0:
        return learning_rate * 0.5
    else:
        return learning_rate

# 創建學習率調度器
lr_callback = LearningRateScheduler(lr_scheduler)
# 早停機制
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0,
                  patience=10, mode='auto')
# 斷點續訓
checkpoint_callback = ModelCheckpoint('checkpoint.ckpt', monitor='val_accuracy',
                    save_weights_only=True, verbose=1,
                    save_best_only=True, mode='auto')
'''
if os.path.exists('checkpoint'):
    model.load_weights('checkpoint.ckpt')
    print("Cotinue with checkpoint")
else:
    print("No checkpoint")
'''
print('Training ---------------------------------')
history = model.fit(train_dataset,
           epochs=100,
           batch_size=128,
           validation_data=val_dataset,
           callbacks=[lr_callback,
                earlystop_callback,
                checkpoint_callback])
model.save('model.h5')

# 繪製圖表
plt.figure()
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc= history.history['val_accuracy']
plt.subplot(2, 1, 1)
plt.title('model loss')
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend(['train_loss',
      'valid_loss'],
      loc='upper right')
plt.subplot(2, 1, 2)
plt.title('model accuracy')
plt.plot(acc, label='acc')
plt.plot(val_acc, label='val_acc')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend(['train_accuracy',
      'valid_accuracy'],
      loc='upper right')
plt.tight_layout()
plt.savefig('./result.png')


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from pretty_confusion_matrix import pp_matrix_from_data
from keras.models import load_model
import numpy as np
import cv2
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

test = ImageDataGenerator(rescale=1/255)
test_dataset = test.flow_from_directory('/content/dataset/test/',
                     target_size=(46,46),
                     batch_size=256,
                     color_mode='grayscale',
                     class_mode='categorical',
                     shuffle=False)

model = load_model('model.h5')
print('\nTesting ------------')
loss, accuracy = model.evaluate(test_dataset)
print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

j=1
k=0
for i in 'down','left','ok','paper','right','rock','scissors','stone','thumb','up':
    x = cv2.imread('/content/dataset/test/{}/{}.jpg'.format(i,j))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
    x = x/255
    x = cv2.resize(x, (46, 46))
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    #print(pred)
    print(np.argmax(pred)+1)
    if np.argmax(pred)+1 == j:
      print('correct')
      k+=1
    else:
      print('false')
    j+=1
print(k)


# 預測測試資料
Y_pred = model.predict(test_dataset)
# 將預測結果轉成類別編號
y_pred = np.argmax(Y_pred, axis=1)
# 取得真實類別編號
y_true = test_dataset.classes
# 繪製混淆矩陣
class_name = ['down',
        'left',
        'ok',
        'paper',
        'right',
        'rock',
        'scissors',
        'stone',
        'thumb',
        'up']
cmap = 'terrain'
pp_matrix_from_data(y_true, y_pred, cmap=cmap, columns=class_name)
plt.savefig('./confusion.png')


In [ ]:
with lq.context.quantized_scope(True):
   model.set_weights(model.get_weights())

for layer in model.layers:
  if 'conv' in layer.name:
    print(layer.name)
    weight, bias = model.get_layer(layer.name).get_weights()
    print(np.shape(weight))
    f = open('./weight/W_'+layer.name+'.bin', 'w')
    for i in range(np.shape(weight)[0]):
      for j in range(np.shape(weight)[1]):
        for k in range(np.shape(weight)[2]):
          for l in range(np.shape(weight)[3]):
            f.write(str(weight[i][j][k][l])+"\n")
    f.close()
    f = open('./weight/b_'+layer.name+'.bin', 'w')
    for i in range(np.shape(bias)[0]):
      f.write(str(bias[i])+"\n")
    f.close()

for layer in model.layers:
  if 'batch_normalization' in layer.name:
    print(layer.name)
    a = model.get_layer(layer.name).get_weights()
    weight = a[0]/pow(a[3]+0.001, 0.5)
    bias = -(a[0]*a[2])/pow(a[3]+0.001, 0.5) + a[1]
    print(np.shape(weight))
    f = open('./weight/W_'+layer.name+'.bin', 'w')
    for i in range(np.shape(weight)[0]):
      f.write(str(weight[i])+"\n")
    f.close()
    f = open('./weight/b_'+layer.name+'.bin', 'w')
    for i in range(np.shape(bias)[0]):
      f.write(str(bias[i])+"\n")
    f.close()

In [ ]:
for layer in model.layers:
  if 'conv' in layer.name:
    weight, bias = model.get_layer(layer.name).get_weights()
    print(layer.name, weight.shape, bias.shape)

In [ ]:
model.summary()